<a href="https://colab.research.google.com/github/pyagoubi/Stuff/blob/main/cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import math
import numpy as np
import matplotlib as plt
from sklearn.linear_model import LinearRegression
import os
import re

In [ ]:
file_names = ['bakery' , 'bar', 'education', 'grocery', 'kiosk', 'post',
               'restaurant', 'veloparking', 'pharmacy', 'public_park',
               'coworking', 'haltestelle']

#file_names = ['haltestelle']

muns_df = pd.read_excel('/content/drive/MyDrive/mun/data/municipalities.xlsx')
muns_df.rename(columns={muns_df.columns[0]: "municipality"}, inplace=True)

# Load gmd dataframe (replace 'gmd.csv' with the actual file or path)
gmd_df = pd.read_excel('/content/drive/MyDrive/mun/data/gmd.xlsx')



def filter_dataframes(gmd_df, file_names):
    save_dir = '/content/drive/MyDrive/mun/data/cleaned_v01/'

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    total_files = len(file_names)
    current_file = 0

    all_dfs = {}  # Dictionary to hold all the dataframes

    for file_name in file_names:
        current_file += 1
        print(f'Processing file {current_file}/{total_files}: {file_name}...')

        df = pd.read_excel(f'/content/drive/MyDrive/mun/data/{file_name}.xlsx')

        df = df[df['Address'].astype(str).str.contains('Switzerland', case=False)]
        df.drop_duplicates(subset=['Name', 'Address'], keep='first', inplace=True)

        clean_df = pd.DataFrame()
        dirty_df = pd.DataFrame()

        dirty_df = df  # Since we're not checking municipalities in address, all rows are considered 'dirty' initially

        drop_indices = []

        for i, row in dirty_df.iterrows():
            for j, gmd_row in gmd_df.iterrows():
                if str(gmd_row['PLZ']) in row['Address']:
                    dirty_df.at[i, 'Municipality'] = gmd_row['Gemeindename']
                    clean_df = pd.concat([clean_df, pd.DataFrame(dirty_df.loc[i]).T], ignore_index=True)
                    drop_indices.append(i)

        dirty_df.drop(drop_indices, inplace=True)

        clean_df.to_excel(save_dir + file_name + '_clean.xlsx', index=False)
        dirty_df.to_excel(save_dir + file_name + '_dirty.xlsx', index=False)

        all_dfs[file_name + '_clean'] = clean_df
        all_dfs[file_name + '_dirty'] = dirty_df

        print(f'Finished processing file {current_file}/{total_files}: {file_name}. Saved the cleaned and dirty versions.')

    return all_dfs

# Run the function and store the dictionary
all_dataframes = filter_dataframes(gmd_df, file_names)






In [7]:
import pandas as pd
import os

# The folder where the clean DataFrames are stored
old_dir = '/content/drive/MyDrive/mun/data/cleaned_v01/'

new_dir = '/content/drive/MyDrive/mun/data/clean_v03/'

# Create new directory to store cleaned files
if not os.path.exists(new_dir):
    os.makedirs(new_dir)

# List of file names
file_names = ['bakery' , 'bar', 'education', 'grocery', 'kiosk', 'post',
               'restaurant', 'veloparking', 'pharmacy', 'public_park',
               'coworking', 'haltestelle']

# Loop over all files
for file_name in file_names:
    print(f'Processing file {file_name}...')

    # Load the clean DataFrame
    clean_df = pd.read_excel(old_dir + file_name + '_cleanv02.xlsx')

    # Remove rows that are duplicated in 'Name' and 'Address', keeping the last one (assuming the correct municipality is the last one)
    clean_df.drop_duplicates(subset=['Name', 'Address'], keep='first', inplace=True)

    # Save the clean DataFrame
    clean_df.to_excel(new_dir + file_name + '_clean_v03.xlsx', index=False)

    print(f'Finished processing file {file_name}. Saved the cleaned version.')


Processing file bakery...
Finished processing file bakery. Saved the cleaned version.
Processing file bar...
Finished processing file bar. Saved the cleaned version.
Processing file education...
Finished processing file education. Saved the cleaned version.
Processing file grocery...
Finished processing file grocery. Saved the cleaned version.
Processing file kiosk...
Finished processing file kiosk. Saved the cleaned version.
Processing file post...
Finished processing file post. Saved the cleaned version.
Processing file restaurant...
Finished processing file restaurant. Saved the cleaned version.
Processing file veloparking...
Finished processing file veloparking. Saved the cleaned version.
Processing file pharmacy...
Finished processing file pharmacy. Saved the cleaned version.
Processing file public_park...
Finished processing file public_park. Saved the cleaned version.
Processing file coworking...
Finished processing file coworking. Saved the cleaned version.
Processing file halt

In [ ]:
!pip install xlsxwriter


In [11]:
import pandas as pd
import os
import glob

import os
import pandas as pd

def combine_excels_in_folder_to_tabs(input_folder, output_filename):
    writer = pd.ExcelWriter(output_filename, engine='openpyxl')

    for filename in os.listdir(input_folder):
        if filename.endswith('.xlsx'):  # Ensure we're dealing with an .xlsx file
            df = pd.read_excel(os.path.join(input_folder, filename))
            # Split the filename at '_clean', take the first part, and use it as the sheet name
            tab_name = filename.split('_clean')[0]
            df.to_excel(writer, sheet_name=tab_name, index=False)

    writer.save()

# usage
combine_excels_in_folder_to_tabs('/content/drive/MyDrive/mun/data/clean_v03/', '/content/drive/MyDrive/mun/data/combinedv03.xlsx')



<ipython-input-11-c7da7292aeab>:18: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [6]:
import glob

def further_clean_all(muns_df, dir_path):
    all_dfs = {}  # Dictionary to hold all the dataframes

    # Load all '_dirty' Excel files from the directory
    all_dirty_files = glob.glob(f"{dir_path}/*_dirty.xlsx")

    for file_path in all_dirty_files:
        # Extract the file name without '_dirty.xlsx'
        file_name = file_path.split('/')[-1].replace('_dirty.xlsx', '')

        # Load the dirty dataframe from the file
        dirty_df = pd.read_excel(file_path)
        # Load the clean dataframe from the file
        clean_df = pd.read_excel(f"{dir_path}/{file_name}_clean.xlsx")

        drop_indices = []

        for i, row in dirty_df.iterrows():
            for municipality in muns_df['municipality']:
                # Check if the municipality is in the 'Address' as a standalone word
                # We use word boundaries (\b) in regex to ensure that we match the municipality as a standalone word
                if re.search(fr'\b{municipality}\b', row['Address']):
                    dirty_df.at[i, 'Municipality'] = municipality  # Replace 'Municipality' field with the matching municipality
                    clean_df = pd.concat([clean_df, pd.DataFrame(dirty_df.loc[i]).T], ignore_index=True)  # Add row to clean_df
                    drop_indices.append(i)  # Add the index to the list of rows to be dropped
                    break  # Exit the loop once we find a match

        # Drop the matched rows from dirty_df
        dirty_df.drop(drop_indices, inplace=True)

        # Save the updated clean dataframe and the remaining dirty dataframe back to disk
        clean_df.to_excel(f"{dir_path}/{file_name}_cleanv02.xlsx", index=False)
        dirty_df.to_excel(f"{dir_path}/{file_name}_dirtyv02.xlsx", index=False)

        # Add the updated clean dataframe and the remaining dirty dataframe to the dictionary
        all_dfs[file_name + '_cleanv02'] = clean_df
        all_dfs[file_name + '_dirtyv02'] = dirty_df

    return all_dfs

# Run the function
dir_path = "/content/drive/MyDrive/mun/data/cleaned_v01"  # Replace with your directory path
all_dfs = further_clean_all(muns_df, dir_path)



In [39]:

def load_excel_files(file_names, directory):
    # Initialize two dictionaries to store dataframes
    clean_dfs = {}
    dirty_dfs = {}

    for file_name in file_names:
        # Construct file paths for clean and dirty data
        clean_file_path = os.path.join(directory, f"{file_name}_clean.xlsx")
        dirty_file_path = os.path.join(directory, f"{file_name}_dirty.xlsx")

        # Load the excel files into pandas DataFrames
        if os.path.exists(clean_file_path):
            clean_dfs[file_name] = pd.read_excel(clean_file_path)
        else:
            print(f"File not found: {clean_file_path}")

        if os.path.exists(dirty_file_path):
            dirty_dfs[file_name] = pd.read_excel(dirty_file_path)
        else:
            print(f"File not found: {dirty_file_path}")

    return clean_dfs, dirty_dfs

clean_dfs, dirty_dfs = load_excel_files(file_names, '/content/drive/MyDrive/mun/data/cleaned_v01')


In [40]:
def print_dataframe_lengths(dfs):
    for name, df in dfs.items():
        print(f"Length of {name}: {len(df)}")

print_dataframe_lengths(dirty_dfs)

Length of bakery: 17
Length of bar: 45
Length of education: 96
Length of grocery: 8
Length of kiosk: 53
Length of post: 12
Length of restaurant: 26
Length of veloparking: 4
Length of pharmacy: 21
Length of public_park: 25
Length of coworking: 2
Length of haltestelle: 17


In [41]:
def combine_excel_files(file_names, directory, output_file):
    with pd.ExcelWriter(output_file) as writer:
        for file_name in file_names:
            # Load the clean and dirty dataframes
            clean_df = pd.read_excel(f"{directory}/{file_name}_clean.xlsx")
            dirty_df = pd.read_excel(f"{directory}/{file_name}_dirty.xlsx")

            # Save them into the excel file with their respective names as the sheet name
            clean_df.to_excel(writer, sheet_name=f"{file_name}_clean", index=False)
            dirty_df.to_excel(writer, sheet_name=f"{file_name}_dirty", index=False)

# Call the function
combine_excel_files(file_names, '/content/drive/MyDrive/mun/data/cleaned_v01', '/content/drive/MyDrive/mun/combined.xlsx')
